# coroutine(协程)理解

从语法上来看，协程和生成器类似，都是定义体中包含yield关键字的函数。但是又不一样,协程重点在流程控制方面

### yield在协程中的用法

* 在协程中yield通常出现在表达式的右边，可以产出值, 例如：`data = yield x`
* 也可以不产出值, 如果yield关键字后面没有表达式，那么生成器产出的值为None,例如: `data = yield`
* 协程可以从调用方接受数据，调用方是通过`send(data)`的方式把数据提供给协程使用，而不是`next(...)`函数，通常调用方会把值推送给协程。
* 协程可以把控制权让给中心调度程序，从而激活其他的协程

所以总体上在协程中把yield看做是**控制流程的方式**。

### 协程运行过程

In [8]:
def simple_coroutine():
    print('---> coroutine started')
    x = yield
    print('---> coroutine received:', x)
    
my_coro = simple_coroutine()
print('定义一个生成器')
print(my_coro)
print('启动激活生成器')
print(next(my_coro))
print('向生成器传递值')
print(my_coro.send(10))

定义一个生成器
<generator object simple_coroutine at 0x000002A80A6D6048>
启动激活生成器
---> coroutine started
None
向生成器传递值
---> coroutine received: 10


StopIteration: 

对上述例子的分析：
* yield 的右边没有表达式，所以这里产出的值是None

* 刚开始先调用了`next(...)`是因为这个时候生成器还没有启动，没有停在yield那里，这个时候也是无法通过send发送数据。所以当我们通过`next(...)`激活协程后，程序就会运行到`x = yield`，这里有个问题我们需要**注意**，`x = yield`这个表达式的计算过程是先计算等号右边的内容，然后在进行赋值，所以当激活生成器后，程序会停在yield这里，**但并没有给x赋值**。

* 当我们调用send方法后yield会收到这个值并赋值给x,而当程序运行到协程定义体的末尾时和用生成器的时候一样会抛出StopIteration异常

如果协程没有通过`next(...)`激活(同样我们可以通过`send(None)`的方式激活)，但是我们直接send，会提示如下错误：

In [10]:
def simple_coroutine():
    print('---> coroutine started')
    x = yield
    print('---> coroutine received:', x)
    
my_coro = simple_coroutine()
print('定义一个生成器')

my_coro.send(12)

定义一个生成器


TypeError: can't send non-None value to a just-started generator

关于调用`next(...)`函数这一步通常称为”预激(prime)“协程，即让协程向前执行到第一个yield表达式，准备好作为活跃的协程使用

### 协程状态

协程在运行过程中有四个状态：

1. GEN_CREATE:等待开始执行
2. GEN_RUNNING:解释器正在执行，这个状态一般看不到
3. GEN_SUSPENDED:在yield表达式处暂停
4. GEN_CLOSED:执行结束

In [24]:
from inspect import getgeneratorstate

def simple_coro(a):
    print('-->started: a=', a)
    b = yield a
    print('-->recevied: b=', b)
    c = yield a+b
    print('-->recevied: c=', c)
    
coro = simple_coro(10)

print(getgeneratorstate(coro))
print(next(coro))
print(getgeneratorstate(coro))
print(coro.send(20))

try:
    coro.send(30)
except StopIteration as e:
    print(e)
    print(e.value)
    
print(getgeneratorstate(coro))

GEN_CREATED
-->started: a= 10
10
GEN_SUSPENDED
-->recevied: b= 20
30
-->recevied: c= 30

None
GEN_CLOSED


想要开始使用协程的时候必须通过`next(...)`方式激活协程，如果不预激，这个协程就无法使用，

如果哪天在代码中遗忘了那么就出问题了，所以有一种预激协程的装饰器，预激装饰器的演示例子::

In [22]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

@coroutine
def averager():
    total = 0.0
    count= 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count
  
coro_avg = averager()
from inspect import getgeneratorstate
print(getgeneratorstate(coro_avg))
print(coro_avg.send(10))
print(coro_avg.send(20))
print(coro_avg.send(30))


GEN_SUSPENDED
10.0
15.0
20.0


关于预激，在使用`yield from`句法调用协程的时候，会自动预激活，这样其实与我们上面定义的coroutine装饰器是不兼容的，在python3.4里面的asyncio.coroutine装饰器不会预激协程，因此兼容yield from

### 中止协程和异常处理

协程中为处理的异常会向上冒泡,传给next函数或send函数的调用方(即触发协程的对象)

如果我们send一个错误的数据就会报错，并且这个时候协程就被终止了,协程状态是GEN_CLOSED.

从python2.5开始客户端代码在生成器对象上调用两个方法: 分别为：throw和close，显示的把异常发送给协程

generator.throw:会让生成器在暂停的yield表达式处抛出指定的异常，
* 如果生成器处理了抛出的异常，代码会向前执行到下一个yield表达式，而产出的值会成为调用generator.throw方法代码的返回值。
* 如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中。

generator.close:会让生成器在暂停的yield表达式处抛出GeneratorExit异常。
* 如果生成器没有处理这个异常，或者抛出了StopIteration异常，调用方不会报错
* 如果收到GeneratorExit异常，生成器一定不能产出值，否则解释器会抛出RuntimeError异常。生成器抛出的异常会向上冒泡，传给调用方。

In [26]:
class DemoException(Exception):
    """定义异常类型"""
    pass

def demo_exc_handling():
    print('--> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('DemoException handled. continuing...')
        else:
            print('-->coroutiine received:{!r}'.format(x))
    raise RuntimeError('this line should never run.')
 

exc_coro = demo_exc_handling()

next(exc_coro)

exc_coro.send(10)

exc_coro.send(20)

exc_coro.send(30)

exc_coro.throw(TypeError)

from inspect import getgeneratorstate

getgeneratorstate(exc_coro)

--> coroutine started
-->coroutiine received:10
-->coroutiine received:20
-->coroutiine received:30


TypeError: 

当传入我们定义的异常时不会影响协程，协程不会停止，可以继续send,但是如果是没有处理的异常的时候，就会报错，并且协程会被终止

### 让协程返回值

In [27]:
from collections import namedtuple

Result = namedtuple('Result', 'colunt average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

coro_avg = averager()

next(coro_avg)
coro_avg.send(10)
coro_avg.send(20)
coro_avg.send(30)

try:
    coro_avg.send(None)
except StopIteration as e:
    result = e.value
    print(result)

Result(colunt=3, average=20.0)


其实上面这种方式获取返回值比较麻烦，而`yield from`结构会自动捕获StopIteration异常，这种处理方式与for循环处理StopIteration异常的方式一样，

循环机制使我们更容易理解处理异常，对于`yield from`来说，解释器不仅会捕获StopIteration异常，还会把value属性的值变成`yield from`表达式的值.

### 关于yield from

* 在生成器中使用`yield from subgen()`时，subgen(子生成器)会获得控制权，把产出的值传给gen的调用方，即**调用方可以直接控制subgen,同时，gen会阻塞，等待subgen终止**

* `yield from x`表达式对x对象所做的**第一件事**是，调用`iter(x)`,从中获取迭代器，因此x可以是任何可迭代的对象

In [28]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i
        
print(list(gen()))

def gen2():
    yield from 'AB'
    yield from range(1, 3)
    
print(list(gen2()))

['A', 'B', 1, 2]
['A', 'B', 1, 2]


这两种的方式的结果是一样的，但是这样看来yield from更加简洁，

但是yield from的作用可不仅仅是替代产出值的嵌套for循环。

yield from的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，

这样二者可以直接发送和产出值，还可以直接传入异常，而不用再像之前那样在位于中间的协程中添加大量处理异常的代码

通过yield from还可以链接可迭代对象

![](images/1.png)

委派生成器在yield from 表达式处暂停时，调用方可以直接把数据发给子生成器，子生成器再把产出产出值发给调用方，子生成器返回之后，解释器会抛出StopIteration异常，并把返回值附加到异常对象上，此时委派生成器会恢复。

In [30]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 子生成器
def averager():
    total, count, average = 0.0, 0, None
    while True:
        term = yield
        if term is None: break;
        total += term
        count += 1
        average = total/count
    return Result(count, average)

# 委派生成器
def grouper(result, key):
    while True:
        result[key] = yield from averager()
        
    
# 客户端代码,即调用方
def main(data):
    results = {}
    for key, values in data.items():
        # 每次循环创建一个grouper,即委派生成器
        group = grouper(results, key)
        print(id(group))
        next(group)
        for value in values:
            group.send(value)
        group.send(None) # 这里表示要中止了
    
    report(results)
    
# 输出报告
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} average {:.2f}{}'.format(result.count, group, result.average, unit))
        
data = {
     'girls;kg':
         [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
     'girls;m':
         [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
     'boys;kg':
         [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
     'boys;m':
         [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

2920752913312
2920752913752
2920752910760
2920752913312
 9 boys  average 40.42kg
 9 boys  average 1.39m
10 girls average 42.04kg
10 girls average 1.43m


关于上述代码着重解释一下关于委派生成器部分，这里的循环每次迭代时会新建一个averager实例，每个实例都是作为协程使用的生成器对象。

grouper发送的每个值都会经由yield from处理，通过管道传给averager实例。

grouper会在`yield from`表达式处暂停，等待averager实例处理客户端发来的值。

averager实例运行完毕后，返回的值会绑定到`results[key]`上，while 循环会不断创建averager实例，处理更多的值.

并且上述代码中的子生成器可以使用return 返回一个值，而返回的值会成为yield from表达式的值。

### 关于yield from的意义

关于yield from 六点重要的说明：

1. 子生成器产出的值都直接传给委派生成器的调用方(即客户端代码)

2. 使用send()方法发送给委派生成器的值都直接传给子生成器。如果发送的值为None,那么会给委派调用子生成器的__next__()方法。如果发送的值不是None,那么会调用子生成器的send方法，如果调用的方法抛出StopIteration异常，那么委派生成器恢复运行，任何其他异常都会向上冒泡，传给委派生成器

3. 生成器退出时，生成器(或子生成器)中的return expr表达式会出发StopIteration(expr)异常抛出

4. yield from表达式的值是**子生成器终止时**传给StopIteration异常的第一个参数。yield from 结构的另外两个特性与异常和终止有关。

5. 传入委派生成器的异常，除了GeneratorExit之外都传给子生成器的throw()方法。如果调用throw()方法时抛出StopIteration异常，委派生成器恢复运行。StopIteration之外的异常会向上冒泡，传给委派生成器

6. 如果把GeneratorExit异常传入委派生成器，或者在委派生成器上调用close()方法，那么在子生成器上调用clsoe()方法，如果它有的话。如果调用close()方法导致异常抛出，那么异常会向上冒泡，传给委派生成器，否则委派生成器抛出GeneratorExit异常